# Exercise 2.5 | Task

##### 1. Import Library and Dataset
##### 2. Creation of aggregated dataframe
##### 3. Initializing instance of kepler.gl map

##### 1. Import Library and Dataset

In [1]:
import pandas as pd
import os
from keplergl import KeplerGl
from pyproj import CRS
import numpy as np
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv('newyork_data_sample.csv', index_col = 0)

C:\Users\repla\AppData\Local\Temp\ipykernel_49392\1723540456.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('newyork_data_sample.csv', index_col = 0)


In [3]:
df.dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
avgTemp               float64
bike_rides_daily        int64
_merge                 object
dtype: object

##### 2. Creation of aggregated dataframe

In [4]:
# Create a new column with the value of 1
df['trip_count'] = 1

In [5]:
# Aggregate the DataFrame to count trips between starting and ending stations
aggregated_df = df.groupby(['start_station_name', 'end_station_name']).agg({'trip_count': 'sum'}).reset_index()

# Display the aggregated DataFrame
aggregated_df.head()

,start_station_name,end_station_name,trip_count
0,1 Ave & E 110 St,1 Ave & E 110 St,27
1,1 Ave & E 110 St,1 Ave & E 44 St,1
2,1 Ave & E 110 St,1 Ave & E 62 St,1
3,1 Ave & E 110 St,1 Ave & E 68 St,2
4,1 Ave & E 110 St,1 Ave & E 78 St,1


In [6]:
# Rename the 'trip_count' column to 'trips'
aggregated_df = aggregated_df.rename(columns={'trip_count': 'trips'})

# Display the updated DataFrame
aggregated_df.head()

,start_station_name,end_station_name,trips
0,1 Ave & E 110 St,1 Ave & E 110 St,27
1,1 Ave & E 110 St,1 Ave & E 44 St,1
2,1 Ave & E 110 St,1 Ave & E 62 St,1
3,1 Ave & E 110 St,1 Ave & E 68 St,2
4,1 Ave & E 110 St,1 Ave & E 78 St,1


In [7]:
print(df.columns)

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'avgTemp', 'bike_rides_daily', '_merge', 'trip_count'],
      dtype='object')


In [8]:
agg_df = df.groupby(['start_station_name', 'end_station_name']).agg({
    'start_lat': 'first',
    'start_lng': 'first',
    'end_lat': 'first',
    'end_lng': 'first',
    'ride_id': 'count'
}).reset_index()

# Rename columns
agg_df = agg_df.rename(columns={
    'start_lat': 'Latitude_x',
    'start_lng': 'Longitude_x',
    'end_lat': 'Latitude_y',
    'end_lng': 'Longitude_y',
    'ride_id': 'trips'
})

In [9]:
agg_df.head()

,start_station_name,end_station_name,Latitude_x,Longitude_x,Latitude_y,Longitude_y,trips
0,1 Ave & E 110 St,1 Ave & E 110 St,40.792310,-73.938156,40.792327,-73.938300,27
1,1 Ave & E 110 St,1 Ave & E 44 St,40.792327,-73.938300,40.750020,-73.969053,1
2,1 Ave & E 110 St,1 Ave & E 62 St,40.792327,-73.938300,40.761227,-73.960940,1
3,1 Ave & E 110 St,1 Ave & E 68 St,40.792327,-73.938300,40.765005,-73.958185,2
4,1 Ave & E 110 St,1 Ave & E 78 St,40.792327,-73.938300,40.771404,-73.953517,1


In [10]:
agg_df = agg_df[['start_station_name', 'end_station_name', 'trips', 'Latitude_x', 'Longitude_x', 'Latitude_y', 'Longitude_y']]

In [11]:
agg_df.head()

,start_station_name,end_station_name,trips,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,1 Ave & E 110 St,1 Ave & E 110 St,27,40.792310,-73.938156,40.792327,-73.938300
1,1 Ave & E 110 St,1 Ave & E 44 St,1,40.792327,-73.938300,40.750020,-73.969053
2,1 Ave & E 110 St,1 Ave & E 62 St,1,40.792327,-73.938300,40.761227,-73.960940
3,1 Ave & E 110 St,1 Ave & E 68 St,2,40.792327,-73.938300,40.765005,-73.958185
4,1 Ave & E 110 St,1 Ave & E 78 St,1,40.792327,-73.938300,40.771404,-73.953517


In [12]:
print("Latitude_x Range:", agg_df['Latitude_x'].min(), agg_df['Latitude_x'].max())
print("Longitude_x Range:", agg_df['Longitude_x'].min(), agg_df['Longitude_x'].max())
print("Latitude_y Range:", agg_df['Latitude_y'].min(), agg_df['Latitude_y'].max())
print("Longitude_y Range:", agg_df['Longitude_y'].min(), agg_df['Longitude_y'].max())

Latitude_x Range: 40.633357763 40.882278085
Longitude_x Range: -74.0836394 -73.881396294
Latitude_y Range: 40.633385 40.88226
Longitude_y Range: -74.0836394 -73.88145


##### 3. Initializing instance of kepler.gl map

In [13]:
import json

In [15]:
from keplergl import KeplerGl
import geopandas as gpd

In [16]:
# Initialize Kepler.gl map instance
map_instance = KeplerGl(height=600,width=800)

gdf=gpd.GeoDataFrame(agg_df,geometry=gpd.points_from_xy(agg_df.Longitude_x,agg_df.Latitude_x))
map_instance.add_data(data=gdf,name="Newyork Data Sample")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [17]:
# Customizing output of map: once I figured out how to add the layers, and have them correlate correctly to one another, it was easy to create them, and dudplicate as
# need be. I picked colours that compliment one another and can be easily distingued. Also, in the 'start' and 'end' point layers, I reduced the radius, as the small size of NYC, 
# plus the sheer volume of instances was basically blocking out the map completely (unless you zoom in quite close)
# In the 'start-end arc', and 'start-end point' layers, I applied the same color plan.. have them be clearly distinguishable against one another. Also, I've reduced the 'stroke'
# setting to just '0.5', as there are so many trips, I wanted to be able to see them without the entire map being coloured out.


In [21]:
# Filters -  Most common trips in NYC: The 3 most common trips are not in a similar area, which I find interesing. One is right beside the bridge that crosses from New Jersey
# over to NYC called 'Pier 40', another in the Lower East Side, and the last is up in Chelsea. 

# The Pier 40 in Hudson River Park is a popular spot for various recreational activities such as sports, outdoor events, and leisurely walks along the waterfront.

# Lower East side location - The areas around Norfolk St & Broome St and Henry St & Grand St in Lower Manhattan are bustling due to a combination of residential density, 
# shopping and dining options, cultural attractions, proximity to transportation hubs, and frequent events and activities. 
# These factors contribute to a vibrant atmosphere with a constant flow of people, both locals and tourists.

# Chelsea - The area around W 22nd St & 8th Ave and W 22nd St & 10th Ave in Chelsea, New York City, is busy due to attractions like the High Line Park, 
# Chelsea Market, art galleries, luxury residential buildings, restaurants, bars, and shopping options, making it a popular destination for locals and tourists alike. 

In [22]:
import json
with open("config.json", "w") as outfile:
    json.dump(config, outfile)